In [ ]:
import os
from calendar import monthrange
import numpy as np
import math
import time
import pandas as pd

In [ ]:
sea_file = "Storm and sea level/data/sea_cropped.csv"
wind_file = "Storm and sea level/data/wind_cropped.csv"
wind_df = pd.read_csv(wind_file, index_col=0)
sea_df = pd.read_csv(sea_file, index_col=0)

Далее представлен основной код для генерирования выборки. Для заданной скорости ветра р итерируемся построчно (то есть находимся в одной точке на карте) и смотрим на изменение уровня моря со временем. Если выполняются нужные условия, то точка добавляется в выборку. В файле данные будут записаны в формате "разность средних, количество штормовых дней, количество спокойных дней, координаты точки, дата, скорость ветра"

In [ ]:
p = 10
windowlen = 11
shp = wind_df.shape
row_names = wind_df.index.values
column_names = wind_df.columns.values

for i in range(shp[0]):
    coord_dots = list()
    j = 0
    while j < shp[1]:
        while wind_df.iloc[i, j] < p and j + 10 < shp[1]:
            j += 1
        if j + 10 >= shp[1]:
            j += 10
            continue
        dot_info = [0, 0, 0, 0]
        for k in range(windowlen):
            if pd.notna(sea_df.iloc[i, j]):
                if wind_df.iloc[i, j] < p:
                    dot_info[2] += sea_df.iloc[i, j]
                    dot_info[3] += 1
                else:
                    dot_info[0] += sea_df.iloc[i, j]
                    dot_info[1] += 1
            j += 1
        j -= 1
        if dot_info[1] >= 4 and dot_info[3] >= 4:
            diff_val = dot_info[0]/dot_info[1] - dot_info[2]/dot_info[3]
            coord_dots.append([diff_val, dot_info[1], dot_info[3], row_names[i], column_names[j], p])
        j += 1
    f = open("output_bigger{}.txt".format(p), "a")
    for elem in coord_dots:
        f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(elem[0], elem[1], elem[2], elem[3], elem[4], elem[5]))
    f.close()
    if i % 1000 == 0:
        print(i)